This notebook cleans the old slice data and assemble them into a xarray object and store as compressed netcdf.

In [1]:
from windwave.defs import Case, Interface2D
import numpy as np
from windwave.prepare import read_p, read_fields
from windwave.phase import extract_phase
import os
from matplotlib import pyplot as plt
import xarray as xr
import pandas as pd
# from windwave.prepare import load_object

%load_ext autoreload
%autoreload 3

ModuleNotFoundError: No module named 'fio'

In [ ]:
case_pre1 = Case(ustar=0.25, Retau=720, Bo=None, g=None, ak=0.2, LEVEL=10, emax=0.3, OUTLEVEL=9, PRECURSOR=True,
                 working_dir='/projects/DEIKE/jiarongw/turbulence/', prefix='curved_fixREtau_precursor_')
case_pre1.tstart = 53

In [9]:

# case7 = Case(ustar=0.25, Retau=720, Bo=200, g=1, ak=0.15, LEVEL=10, emax=0.3, alterMU=16, NOMATCH=True, 
#              PRINTWAVE=True, OUTLEVEL=9, working_dir='/home/ctrsp-2024/jiarongw/outputs/oldruns/', prefix='curved_fixREtau_boundary_')
# case7.tstart = 43

case = Case(ustar=0.25, Retau=720, Bo=200, g=1, ak=0.15, LEVEL=10, emax=0.3, alterMU=16, NOMATCH=True, 
             PRINTWAVE=True, OUTLEVEL=9, working_dir='/home/ctrsp-2024/jiarongw/outputs/oldruns/', prefix='curved_fixREtau_boundary_')
case.tstart = 44

case.tsimu = np.arange(44.1,60,0.1)
extract_phase(case, case.tsimu)
case.field_t = np.arange(44.5,60,0.5) - case.tstart
read_fields(case)

TypeError: __init__() got an unexpected keyword argument 'mu'

In [ ]:
"""NOTICE: to accomodate different pickle versions"""
from ww.prepare import load_object

tsimu = 54.5

ux_3D = {'name':'ux', 'value':[]}
uy_3D = {'name':'uy', 'value':[]}
# uz_3D = {'name':'uz', 'value':[]} # Alas we didn't save uz_3D 
# f_3D = {'name':'f', 'value':[]}
pair_3D = {'name':'pair', 'value':[]}

da_list = []
for field in (ux_3D, uy_3D, pair_3D):
    
    picklename = case.path + 'field/' + 'pickle_tiger/' + field['name'] +'_t%g' % tsimu +'.pkl'
    exists = os.path.exists(picklename)
    
    # If the pickle is there read in the pickles
    if exists:
        field['value'] = load_object(picklename)
        print('pickle restored!')
    
    x = np.linspace(-np.pi, np.pi, 512, endpoint=False) + 2*np.pi/512/2
    y = np.linspace(0, 2*np.pi, 512, endpoint=False) + 2*np.pi/512/2
    z = np.linspace(-np.pi, np.pi, 128, endpoint=False) + 2*np.pi/128/2
    
    data = field['value'][::2, :, :]  # Saved 255 slices but only take half of it 128 slices
    da = xr.DataArray(data, dims=['z', 'x', 'y'], coords={'x': x, 'y': y, 'z': z})
    field['da'] = da
    
# Convert xarray DataArray to pandas DataFrame
# df = da.to_dataframe(name='ux').reset_index()